In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")
from sklearn.model_selection import train_test_split
import shutil


In [2]:
test_data = pd.read_csv("./Data/Mawqif_AllTargets_Blind Test.csv")
test_data = test_data.fillna("None")
test_data

,ID,text,target
0,2,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...,Women empowerment
1,5,الفساد الإداري انهك البلاد ومازلنا نعتمد عل...,Digital Transformation
2,14,-البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...,Covid Vaccine
3,16,منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...,Digital Transformation
4,17,انا مع تمكين المرأة اصلا URL,Women empowerment
...,...,...,...
614,4097,التحول الإلكتروني دا معاه مخاطر إلكترونية كبير...,Digital Transformation
615,4098,ريم النجم: تطعيم كورونا راح يحولكم زومبي !,Covid Vaccine
616,4099,البيانات هي العنصر الرئيسي و المهم جداً في رحل...,Digital Transformation
617,4102,تطعيم انفلونزا مناعة مؤقته تطعيم كورونا مناعة...,Covid Vaccine


In [3]:
test_set_dict = {item['ID']: item for item in test_data.to_dict(orient="index").values()}

In [5]:
mapper={'Women empowerment':'تمكين المرأة',
'Covid Vaccine':'لقاح كوفيد',
'Digital Transformation':'التحول الرقمي'}

In [ ]:
import ollama
from tqdm import tqdm_notebook as tqdm


model_name = "command-r:35b-v0.1-q8_0"
system_context = "You are an expert in analysing people's opinions. You are an expert in Arabic. \
# You will be given an Arabic sentence as input. Your task is to identify the stance towards the topic or subject discussed in the sentence. \
# Your task is to identify whether the sentence is in favour of the topic, against it, or neutral. \
Your output should be one of the following: favour, against, or neutral. You should not provide any further information. Your answer should be in English."
question = "What is the stance in the given sentence? [favour, against, neutral]."

counter = 0
progress_bar = tqdm(test_set_dict.keys(), desc="Processing")
for k in progress_bar:
    text = test_set_dict[k]['text']
    topic = test_set_dict[k]['target']
    msg=f"الجملة: {text}\nالموضوع:{mapper[topic]}\n{question}"
    
    message=[{
        'role': 'system',
        'content':system_context},
        {
            'role':'user',
            'content': msg
        }]
    response = ollama.chat(model=model_name, messages=message)
    #print(response['message']['content'])
    #print('-'*20)
    test_set_dict[k]['label'] = response['message']['content']
    progress_bar.update(1)
progress_bar.close()



In [ ]:
output_ = ["ID\tTarget\tTweet\tStance"]
for k in test_set_dict.keys():
    if 'favour' in test_set_dict[k]['label'].lower():
        label = "favor".upper()
    elif 'against' in test_set_dict[k]['label'].lower():
        label = "against".upper()
    else:
        label = "None".upper()
    output_.append(f"{test_set_dict[k]['ID']}\t{test_set_dict[k]['target']}\t{test_set_dict[k]['text']}\t{label}")


In [ ]:
with open("output_testset_model.csv", "w", encoding="utf-8") as fout:
    for line in output_:
        fout.write(f"{line}\n")